In [1]:
from bs4 import BeautifulSoup
from openai import OpenAI
import requests
import mysql.connector
import re  
import json
from datetime import datetime  

from cnsenti import Sentiment

In [2]:
# 创建数据库连接
db = mysql.connector.connect(
    host="localhost",  # MySQL服务器地址
    user="root",   # 用户名
    password="",  # 密码
    database="soybean"  # 数据库名称
)

# 创建游标对象，用于执行SQL查询
cursor = db.cursor()

In [3]:
# gpt client set up
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-X9zKww2VlHkICfI3Bz1suRX3lTlgm5TxfFehabn1oI2HA8wj",
    base_url="https://api.chatanywhere.tech/v1"
)


def gpt_35_api_stream(messages: list):
    """为提供的对话消息创建新的回答 (流式传输)

    Args:
        messages (list): 完整的对话消息
    """
    stream = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=messages,
        stream=True,
    )
    str = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
#             print(chunk.choices[0].delta.content, end="")
            str += chunk.choices[0].delta.content
    return str

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False

In [4]:
# msg = "【帮我判断下面这段材料，对于豆粕价格是上涨/下跌/趋稳的概率，只给我一个0到1的数字，0表示偏向下跌，0.5表示趋稳，1表示偏向上涨。】"
# msg += "作者：东海期货 刘兵 近日，针对加拿大政府近日宣布对自华进口电动汽车、钢铝产品采取加征关税等限制措施，中方将对加拿大采取的相关限制措施发起“反歧视调查”，后续根据实际情况对加采取相应措施。目前中方对自加拿大进口油菜籽(5208, 89.00, 1.74%)等发起反倾销调查。 回顾近年来中加关系发展 2018年12月，孟晚舟在加拿大被扣押以来，中加关系持续紧张，并直接影响了双边的贸易往来，特别是在菜籽进口方面。2019年3月，由于政治紧张，且中国海关多次在加拿大进口的油菜籽中检出危险性有害物质，导致中国海关作出暂停进口的决定，并取消了部分加拿大油菜籽出口公司的出口许可证。此后，中国开始对加拿大菜籽实行相应进口限制，提高了对菜籽中杂质的限制要求。2021年9月，孟晚舟获释放，期间加中关系一直处于严重冻结状态，此后双边关系也几乎没有起色。2022年5月，中国取消了三年前对加拿大油菜籽的进口限制。2023年5月中加爆发外交事件，互相驱逐外交官，关系再度紧张。直至今年7月，加拿大外长乔利久违地访华，中加关系才初步改善。 近年来中加关系恶化对菜系进口的影响 2019年中国进口菜籽数量骤降42%至274万吨，其中对加拿大菜籽进口占比降至86%。2020年中国进口从俄罗斯补充进口增加75%约14.25万吨菜籽，全年中国菜籽进口同比增加了13%至311万，对加拿大菜籽进口依赖减至74%。2021年俄罗斯菜籽面积大幅扩增，但因干旱影响且提高了菜籽出口关税，导致中国直接进口菜籽数量继续下滑至265万吨。2022年俄乌战争导致港口封锁，中国进口俄菜籽停滞，同年 5月中国取消了对加拿大油菜籽的进口限制，全年中国直接进口菜籽数量再次下滑至196万。自2022年底至2023年上半年，中国进口加拿大菜籽出现爆发时增长， 2023年5月，中加关系再度紧张，同时因2022年加拿大菜籽减产导致2023年供应减少，菜籽进口受阶段性影响增加，但对2023/24年菜籽继续维持大规模采购。据统计，2023年中国全年采购菜籽同比增加180%至549万，其中加拿大菜籽进口占比92%，俄罗斯菜籽进口占比7%。 2024年以来，国内一季度进口菜籽利润丰厚，菜籽全年买船大幅增加。截至今年7月统计，中国累计进口菜籽283万吨，其中93.6%来自加拿大，5.5%来自俄罗斯。2023/24作物年度内，我们预计中国将进口菜籽525万吨，同比减少1.68%；预计中国直接进口菜油(8340, 2.00, 0.02%)或同比相当，约200万吨；直接进口菜籽粕280万，同比增加38.6%或77吨。2022年以来，中国对加拿大菜油直接进口量大幅减少，对俄罗斯直接进口的非转菜油量大幅增加。今年至7月统计，中国进口的菜油中58%来自俄罗斯，15.4%来自阿联酋，尽管对加拿大菜油进口微乎其微，但中国菜粕(2479, 38.00, 1.56%)的进口对加拿大的依赖程度却稳定在75%左右，另外将近18%菜粕进口来源于阿联酋。 中加关系再恶化对国内菜系供应可能的影响 加拿大菜籽进口以大厂浸出法压榨为主，出油率一般在42%，出粕率为56%左右。俄罗斯进口非转菜籽以作坊式压榨法为主，产浓香菜油，参考国内非转菜籽出油率约36%。总体看，2023/24年度中国进口菜籽压榨将产出菜粕275万吨菜粕（不含菜籽饼），217万吨菜油。结合国内直接进口供应量，2023/24年度直接进口且含进口菜籽压榨生产的菜粕量为555万， 进口直接且含进口菜籽压榨生产的菜油417万。 2019年-2021年，国内年平均进口加拿大菜籽为237万吨，比预期的2024年进口少了288万吨，大体折国内菜粕160万菜粕和120万吨菜油；2019-2021年年国内平均进口加拿大菜油95万吨、菜粕150万吨。 假设2024/25作物年度内，中国临时或反倾销调查结束后通过增加关税等措施限制菜籽进口，我们预计对进口影响不会超过2019-2021年平均水平。假设严重至此，菜油方面，国内至少可以通过恢复对加拿大的直接进口，以及增加俄罗斯进口来填补100万吨的缺口；菜粕方面，目前直接从加拿大直接进口量已是历史最高水平，来自阿联酋菜粕的进口供应同水平同2019-2021年大体相当。若如此，预期的菜粕缺口要远大于菜油。总体来看，若事态发展严重到2019-2021年水平，因限制菜籽进口而导致的菜油同2023/24年度内的缺口大体不会超过5%，但菜粕同2023/24年度供应的缺口预计将达到28%。如此情况下，叠加国产菜籽大榨作为补充供应，菜粕预期可供应量将350万吨左右，基本可以满足国内菜粕的刚需以及菜油缺口。如果菜粕缺口消费完全被豆粕(3111, 18.00, 0.58%)替代，预期全年将增加200万大豆(4521, -21.00, -0.46%)压榨，届时产出的豆油(7630, 36.00, 0.47%)将会是油脂额外增量。 中加关系恶化对油脂油料未来行情的影响 今年国内菜系整体供应充足，若事态严发，短期国内菜系供应依然可以维持稳定，中长期看菜粕菜粕刚需完全也可以满足，且可以通过豆粕维持蛋白供应稳定，菜油直接进口替代空间很大，且相关油脂可替代补充量也完全充足。此外，今年全球油料丰产既定，2024/25年度丰产交易趋向也未变，预计油脂油料及蛋白整体价格或不会因为菜系局部风险而出现单边行情。 短期，菜系风险溢价主导的菜棕价差、菜豆粕价差或会出现低估值修复行情。中期，油脂因油料丰产，以棕榈(7878, 112.00, 1.44%)油决定的油脂价格重心的趋势不变；国内豆粕供应预计十分充足，依然是决定蛋白价格重心的核心。远期，重点关注新季度天气可能引发的油料油脂较预期减产的风险，若出现减产交易趋向，菜系或相比豆系及棕榈油更具有弹性空间。"
# msg_text = "【帮我判断下面这段材料，对于豆粕价格是上涨/下跌/趋稳的概率，如果无法判断就说无法判断，只给一句话不超过20字。】"
# msg_text += "读这个链接：https://finance.eastmoney.com/a/202409053176593116.html"

# # messages = [{'role': 'user','content': msg},]

# # call def gpt
# # gpt_value = gpt_35_api_stream(messages)
# # print(gpt_value)

# messages = [{'role': 'user','content': msg_text},]
# gpt = gpt_35_api_stream(messages)
# print(gpt)


In [5]:
# visit the home page for M (Sina)
res = requests.get('https://finance.sina.com.cn/money/future/M/')
res.encoding = res.apparent_encoding
soup = BeautifulSoup(res.text, 'html.parser')

In [8]:
link_list = []
title_list = []
context_list = []
date_list = []
i = 0

li_tags = soup.find_all('li')

for li_tag in li_tags:
    # pass the top 2 news (fake news)
    i +=1
    if i<=2:
        continue
        
    # fetch link and title
    link = li_tag.find('a')['href']
    title = li_tag.text
    
    # check the date of report
    date_pattern = re.compile(r'(\d{2}-\d{2})')  
    current_year = datetime.now().year  
    
    matches = date_pattern.findall(title).pop()
    full_date = f"{current_year}-{matches}"  
    
    # 将字符串转换成日期对象  
    date_obj = datetime.strptime(full_date, '%Y-%m-%d')  
    today_obj = datetime.now().date()  
    is_same_dayss = (date_obj.date() == today_obj)  
    
    # only today news
    if is_same_dayss==False:
        continue
#     if i>20:
#         continue
    
    # get the article text
    text = ""
    link_res = requests.get(link)
    link_res.encoding = link_res.apparent_encoding
    link_soup = BeautifulSoup(link_res.text, 'html.parser')
    try:
        text = link_soup.find(class_="article").get_text()
    except:
        print(title)
    
    # append to list
    link_list.append(link)
    title_list.append(title)
    context_list.append(text)
    date_list.append(full_date)
    
    

[豆粕]大宗商品到"变盘时刻"？高盛最新观点来了！(09-09 16:52)


In [9]:
msg_text = "【帮我判断下面这段材料，对于豆粕价格是上涨/下跌/趋稳的概率，只给一句话不超过20字。】"

gpt_list = []
gpt_value_list = []

for n in range(0,len(link_list)):
    # remove token less than 100
    if len(context_list[n]) <= 20:
        gpt_list.append("")
        continue
    
    #check if token more than 4000 remove the last
    new_context = context_list[n]
    if len(context_list[n]) > 3200:
        new_context = context_list[n][:-(len(context_list[n])-3200)]
    
    # find gpt text
    messages_text = [{'role': 'user','content': msg_text+new_context},]
    gpt = gpt_35_api_stream(messages_text)
    gpt_list.append(gpt)
    print(gpt)


豆粕价格下跌的概率较大。
豆粕价格趋稳的概率较高。
豆粕价格下跌的概率较大。
豆粕价格可能上涨，降雨频繁可能影响产量。
对豆粕价格趋势影响不确定，需独立判断。
豆粕价格下跌的概率较大。
豆粕价格可能下跌。
鉴于生猪行情保持景气度，豆粕价格可能上升。
豆粕价格有上涨的概率。
根据上述材料，对于豆粕价格是趋稳的概率较高。
豆粕价格有上涨的概率。
豆粕价格可能下跌。
豆粕价格趋稳可能性较大。
豆粕价格趋稳的概率较大。
豆粕价格有上涨的概率。
豆粕价格有上涨可能。
下跌概率较大。
豆粕价格趋稳的概率较大。
豆粕价格可能趋稳，受强降雨天气和高温影响。
豆粕价格趋势上涨的概率较大。
豆粕价格下跌的概率较大。
无法判断豆粕价格走势。
豆粕价格有上涨的概率。
豆粕价格有上涨的概率。
豆粕价格有下跌的概率。
豆粕价格趋稳的概率较高。
豆粕价格有上涨的概率。


In [10]:
# fetch the value of gpt text
gpt_value_list = []

for n in range(0,len(gpt_list)):
    senti = Sentiment(pos='pos.txt',  
                      neg='neg.txt', 
                      merge=True,  
                      encoding='utf-8')

    result = senti.sentiment_count(gpt_list[n])
    if result['pos']>result['neg']:
        gpt_value_list.append(1)
    elif result['pos']<result['neg']:
        gpt_value_list.append(0)
    else:
        gpt_value_list.append(0.5)
    print(gpt_list[n])
    print(gpt_value_list[n])
 

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\bleve\AppData\Local\Temp\jieba.cache
Loading model cost 0.622 seconds.
Prefix dict has been built successfully.



0.5
豆粕价格下跌的概率较大。
0
豆粕价格趋稳的概率较高。
0.5
豆粕价格下跌的概率较大。
0
豆粕价格可能上涨，降雨频繁可能影响产量。
0.5
对豆粕价格趋势影响不确定，需独立判断。
1
豆粕价格下跌的概率较大。
0
豆粕价格可能下跌。
0
鉴于生猪行情保持景气度，豆粕价格可能上升。
0.5
豆粕价格有上涨的概率。
1
根据上述材料，对于豆粕价格是趋稳的概率较高。
0.5
豆粕价格有上涨的概率。
1
豆粕价格可能下跌。
0
豆粕价格趋稳可能性较大。
0.5
豆粕价格趋稳的概率较大。
0.5
豆粕价格有上涨的概率。
1
豆粕价格有上涨可能。
1
下跌概率较大。
0
豆粕价格趋稳的概率较大。
0.5
豆粕价格可能趋稳，受强降雨天气和高温影响。
0.5
豆粕价格趋势上涨的概率较大。
1
豆粕价格下跌的概率较大。
0
无法判断豆粕价格走势。
0
豆粕价格有上涨的概率。
1
豆粕价格有上涨的概率。
1
豆粕价格有下跌的概率。
0
豆粕价格趋稳的概率较高。
0.5
豆粕价格有上涨的概率。
1


In [11]:
# insert into database
for n in range(0,len(title_list)):
    sql = "INSERT INTO news (title, link,news_date,context,gpt,gpt_value) VALUES (%s, %s, %s,%s, %s, %s)"
    values = (title_list[n], link_list[n],date_list[n],context_list[n],gpt_list[n],float(gpt_value_list[n]))

    cursor.execute(sql, values)
    
# 提交更改到数据库
db.commit()

In [12]:
# for eastmoney news
page_index = 1
page_size = 20

url = 'https://np-listapi.eastmoney.com/comm/web/getNewsByScColumns?client=web&biz=web_news_sccol&scColumns=278&order=1&page_index='+str(page_index)+'&page_size='+str(page_size)+'&req_trace=1725761781811&fields=code,showTime,title,mediaName,summary,realSort,image,url,uniqueUrl,Np_dst&types=1,20&callback=jQuery18306420563401369495_1725761781759&_=1725761781812'

res = requests.get(url)
res.encoding = res.apparent_encoding

# convert to json
cleaned_res = re.sub(r'jQuery\d+_\d+\(', '', res.text)  
cleaned_res = cleaned_res.rstrip(');')  # 去除末尾的');'
res_json = json.loads(cleaned_res)


In [13]:
# fetch news
link_list = []
title_list = []
context_list = []
date_list = []

link = res_json['data']['list'][0]['url']

for item in res_json['data']['list']:
    
    # 将字符串转换成日期对象  
    today_obj = str(datetime.now().date())
    is_same_dayss = (item['showTime'].split(' ')[0] == today_obj)  
    
    # only today news
    if is_same_dayss==False:
        continue
    
    link = item['url']
    title = item['title']
    full_date = item['showTime'].split(' ')[0]

    # get the article text
    link_res = requests.get(link)
    link_res.encoding = link_res.apparent_encoding
    link_soup = BeautifulSoup(link_res.text, 'html.parser')
    text = link_soup.find(class_="txtinfos").get_text()
    
    # append to list
    link_list.append(link)
    title_list.append(title)
    context_list.append(text)
    date_list.append(full_date)


In [14]:
msg_text = "【帮我判断下面这段材料，对于豆粕价格是上涨/下跌/趋稳的概率，只给一句话不超过20字。】"

gpt_list = []

for n in range(0,len(link_list)):
    # remove token less than 100
    if len(context_list[n]) <= 20:
        gpt_list.append("")
        continue
    
    #check if token more than 4000 remove the last
    new_context = context_list[n]
    if len(context_list[n]) > 3200:
        new_context = context_list[n][:-(len(context_list[n])-3200)]
    
    # find gpt text
    messages_text = [{'role': 'user','content': msg_text+new_context},]
    gpt = gpt_35_api_stream(messages_text)
    gpt_list.append(gpt)
    print(gpt)

In [15]:
# fetch the value of gpt text
gpt_value_list = []

for n in range(0,len(gpt_list)):
    senti = Sentiment(pos='pos.txt',  
                      neg='neg.txt', 
                      merge=True,  
                      encoding='utf-8')

    result = senti.sentiment_count(gpt_list[n])
    if result['pos']>result['neg']:
        gpt_value_list.append(1)
    elif result['pos']<result['neg']:
        gpt_value_list.append(0)
    else:
        gpt_value_list.append(0.5)
#     print(gpt_list[n])
#     print(gpt_value_list[n])
 

In [16]:
# insert into database
for n in range(0,len(title_list)):
    sql = "INSERT INTO news (title, link,news_date,context,gpt,gpt_value) VALUES (%s, %s, %s,%s, %s, %s)"
    values = (title_list[n], link_list[n],date_list[n],context_list[n],gpt_list[n],float(gpt_value_list[n]))

    cursor.execute(sql, values)
    
# 提交更改到数据库
db.commit()